# Object Detection Using Jetson Inference

The following module translates the team's efforts to detect objects using the jetson inference module provided by Nvidia in their "Hello AI World" series of open source online courses. It assumes that pytorch and torchvision has been installed to the OS.

## Setting Up The Camera 

To make sure, that your CSI camera works correctly; ssh into your jetson nano or open a terminal in your wire connected desktop and run the following command.

`nvgstcapture-1.0`

If you can see a live camera feed, this means that the CSI camera was setup correctly and is now ready for further use. 

#### Common Mistakes and How To Fix Them:

1) No feed is seen on the monitor: Make sure that the tail of the CSI camera is properly connected to the jetson board. Use screws to tighten the pocket on the board if needed.

2) Error with GStreamer/Image Buffer: The GStreamer pipelines you are using aren´t exactly the same, they differ in the caps and elements used. For example, framerate and resolution are different.You may try the following GStreamer pipeline (based on your opencv snippet) with gst-launch-1.0 to check if it runs: 
`gst-launch-1.0 -v nvarguscamerasrc ! 'video/x-raw(memory:NVMM),width=1280, height=720, framerate=60/1, format=NV12' ! nvvidconv flip-method=0 ! 'video/x-raw, width=1280, height=720, format=BGRx' ! videoconvert ! 'video/x-raw, format=BGR' ! identity silent=false ! fakesink -e`


## Setting Up Jetson Inference

Follow the followig instructions to successfully setup the jetson inference module and its dependencies on your OS. This will be used in all the tasks that would be done after. Reference: [Building Project from Source](https://github.com/dusty-nv/jetson-inference/blob/master/docs/building-repo-2.md)

`$ sudo apt-get update
$ sudo apt-get install git cmake libpython3-dev python3-numpy
$ git clone --recursive https://github.com/dusty-nv/jetson-inference
$ cd jetson-inference
$ mkdir build
$ cd build
$ cmake ../
$ make -j$(nproc)
$ sudo make install
$ sudo ldconfig
`

By running the above commands, you have now successfully cloned the repo, built, configured and complied the project from the cmake file and downloaded the necessary models.

#### Common Mistakes and How To Fix Them:
1) Cmake was not installed: Make sure that the cmake has been preinstalled. Use `cmake --version` to confirm and if need be run `sudo pip install cmake` to install cmake.

2) Not all models were downloaded: When the below pop up box shows up, make sure that you download all the models and not just a selected few. All of these models would be required for the DetectNet object detection model that we will be using in this task.
<img src= "download-models.jpg">

## Collecting your Dataset

Assuming that your camera has been setup correctly and a live feed from the camera is now accessible, we are now free to collect our custom dataset for the object detection. To collect the images with the bounding boxes for detection, we will be using NVidia's pre built dialog box that can be triggered using the `camera-capture` feature.

Navigate to the ssd folder: `cd jetson-inference/python/training/detection/ssd`

Run the camera capture feature: `camera-capture csi://0`

The below pop up box opens on the desktop:
<img src= "pytorch-collection-detection-widget.jpg">

You should now set the dataset path of the annotations folder you would like to store the annotated images in.

You should now create a .txt file called "labels.txt" in the above folder and write (on different lines) the names of the objects to be detected. Set the labels path to the labels.txt file

You must set the Dataset Type to Detection.

You can toggle between train and test as the Current Set depending on your requirement for quantity of data for your model's training and testing.

To make saving simpler, select the tick box named "Save on Unfreeze" and clear the tick box named "Clear on Unfreeze"


### Marking your Images

Bring the objects to be annotated closer to the camera and click on `Freeze/Edit (space)`. Draw a bounding box around the object and select the class of the object.

Click on the `Freeze/Edit (space)` button again. Your annotated image has now been saved in the dataset path folder. 

You can view the images by navigating to the folder. The images are stored in the Annotations folder inside the dataset folder.

The tool uses Pascal VOC format for storing data which is also the format that our model requires.


#### Common Mistakes and How To Fix Them:
1) Issue with gstreamer while running camera-capture: Navigate to the ssd folder and try running the command `camera-capture csi://0 --input-flip=rotate-180` 

2) Bounding boxes are loose: Try drawing the bboxes as compact as possible for better accuracy of the model. 

## Training Your Model on Your Newly Collected DataSet

We will be training a new model using the `train_ssd.py` file after which the model would be loaded on to [DetectNet](https://github.com/dusty-nv/jetson-inference/blob/master/docs/pytorch-ssd.md) which employs CNN to detect object and non-maximal suppression.

To train your model cd into your ssd folder and run the command `python3 train_ssd.py --dataset-type=voc --data=data/<YOUR-DATASET> --model-dir=models/<YOUR-MODEL>` 

After training you'll need to convert your PyTorch model to ONNX:
`python3 onnx_export.py --model-dir=models/<YOUR-MODEL>`
The converted model will then be saved under `<YOUR-MODEL>/ssd-mobilenet.onnx`, which you can then load with the detectnet program:

`NET=models/<YOUR-MODEL>`

`detectnet --model=$NET/ssd-mobilenet.onnx --labels=$NET/labels.txt \
          --input-blob=input_0 --output-cvg=scores --output-bbox=boxes \
            csi://0
`

You can now detect the objects by using the live camera feed and the accuracy is shown on the edge of the box.